In [1]:
""" Python Standard Library """
import os
import datetime as dt

import json
import logging
""" Third Party Imports """
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_create_report
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

outdir = setup("variant_share_qc.ipynb")

/Users/brittneycontreras/PycharmProjects/covid-models


2022/11/15 14:54:38 INFO|============================================================


In [2]:
import pandas as pd
obs_fpath = "covid_model/model_dev_qc/variants/CDPHE Variant Data (manually transcribed and limited to model variants)_ScrabbleVariant.csv"
obs = pd.read_csv(obs_fpath, parse_dates=['Date']).fillna(0)
obs['Date'] = [d.date() for d in obs['Date']]
obs = obs.set_index('Date')

In [3]:
obs = obs.drop(obs.columns[9:16], axis=1)

In [5]:
seeds = [
    {"param": "wildtype_seed", "attrs": None, "vals": {"2020-01-01": 0, "2020-01-24": 2.2, "2020-01-30": 0}, "desc": "2.2 wildtype seeds are planted each day in CO starting January 24th, 2020"},
    {"param": "alpha_seed", "attrs": None, "vals": {"2020-01-01": 0, "2020-11-09": 1, "2020-12-04": 0}, "desc": "1 Alpha seed is planted each day starting November 9th, 2020 and ending December 4th, 2020"},
    {"param": "delta_seed", "attrs": None, "vals": {"2020-01-01": 0, "2021-03-07": 1, "2021-04-11": 0}, "desc": "1 Delta seed is planted each day starting March 7th, 2021 and ending April 11th, 2021"},
    {"param": "omicron_seed", "attrs": None, "vals": {"2020-01-01": 0, "2021-10-28": 4, "2021-12-10": 0}, "desc": "1 BA.1 seed is planted each day starting November 9th, 2021 and ending December 11th, 2021"},
    {"param": "ba2_seed", "attrs": None, "vals": {"2020-01-01": 0, "2021-12-30": 10, "2022-01-08":5, "2022-01-10": 0}, "desc": "5 BA.2 seeds are planted each day starting December 23rd, 2021 and ending February 2nd, 2022"},
    {"param": "ba2121_seed", "attrs": None, "vals": {"2020-01-01": 0, "2022-01-25": 6, "2022-01-26":0}, "desc": "5 BA.2.12.1 seeds are planted each day starting February 27th, 2022 and ending April 8th, 2022"},
    {"param": "ba45_seed", "attrs": None, "vals": {"2020-01-01": 0, "2022-03-10": 1, "2022-03-15":0}, "desc": "5 BA.4/5 seeds are planted each day started April 5th, 2022 and ending May 15, 2022"},
    {"param": "scrabble_seed", "attrs":  None, "vals":  {"2020-01-01": 0, "2022-09-10": 2, "2022-10-15": 0}, "desc": "baseline Variant X seeding value for scenarios"},
]

model_args = {
    'params_defs': json.load(open('covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json')) + seeds,
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_vacc_proj_params.json',
    'regions': ['co'],
    'mobility_mode': None,
    'start_date': '2020-01-24',
    'end_date': '2023-02-01',
    'max_step_size': 1.0,
    'ode_method': 'RK45',
    'hosp_reporting_frac': {"2020-01-01": 1, "2022-03-01": 0.932, "2022-03-15": 0.868, "2022-03-30": 0.8}
}
#model_args = {'base_spec_id': 4019, # input the spec ID for the model you want here to avoid having to fit it again
              #'update_data': False}
fit_args = {
    #'fit_start_date': '2022-01-24',
    #'fit_end_date': '2022-08-30',
    'tc_min': 0.0,
    'tc_max': 0.999,
    'tc_window_size': 14,
    'tc_window_batch_size': 6,
    'tc_batch_increment': 2,
    'last_tc_window_min_size': 14,
    'outdir': outdir
}
model = CovidModel(**model_args)
model.prep(outdir=outdir)

2022/11/15 14:58:09 INFO|------------------------|{} Retrieving vaccinations data
2022/11/15 14:58:11 INFO|------------------------|{} Constructing vaccination projections
2022/11/15 14:58:12 INFO|------------------------|{} Retrieving hospitalizations data
2022/11/15 14:58:14 INFO|----------------------|{} Prepping Model


In [ ]:
#model.solve_seir()
model = do_single_fit(**fit_args, base_model = model, prep_model=False)

In [ ]:
model.solve_seir()
soln = model.solution_sum_df(['seir', 'variant'])

In [ ]:
start_date = dt.datetime.strptime('2020-02-01', '%Y-%m-%d').date()
end_date = dt.datetime.strptime('2022-11-01', '%Y-%m-%d').date()
rows = (soln.index.get_level_values(0) >= start_date) & (soln.index.get_level_values(0) <= end_date)
cols = [any(colname == seir for seir in ['Ih', 'I', 'A']) for colname in soln.columns.get_level_values(0)]
infected_by_variant = soln.loc[rows, cols].groupby('variant', axis=1).sum()

In [ ]:
infected_by_variant.plot(figsize=(10, 5), cmap='Set2')

In [ ]:
infected_by_variant.div(infected_by_variant.sum(axis=1), axis=0).plot(figsize=(10, 5), cmap='Set2')

In [ ]:
# Saves data from plot above to .csv file - may need to update file path and name depending on preferences
new_infected_by_variant = infected_by_variant.div(infected_by_variant.sum(axis=1), axis=0)
new_df = pd.DataFrame(new_infected_by_variant)
new_df.to_csv('./covid_model/output/norm_variant_prev.csv')

In [ ]:
obs_rows = (obs.index.values >= start_date) & (obs.index.values <= end_date)
obs['Omicron BA.4/5'] = obs['BA.4'] + obs['BA.5']
obs = obs.drop(columns=['BA.4', 'BA.5']).loc[obs_rows,]
obs.plot(figsize=(10,5), cmap='Set2')


In [ ]:
obs.info()

In [ ]:
infected_by_variant.sum(axis=1).plot(figsize=(10, 5))

In [ ]:
model.modeled_vs_observed_hosps().plot(figsize=(10,5), cmap='Set2')

In [ ]:
#do_create_report(model, outdir)

### Make figure plot comparing CDPHE data and model variant share for each variant separately

In [ ]:
# Clean up obs data under new df (new_obs) to make plotting easier

# Normalize observed data
new_obs = obs.div(obs.sum(axis=1), axis=0)
# Get rid of seperate BA.4 and BA.5 columns
#new_obs = new_obs.drop(columns=['BA.4', 'BA.5'])
# Change column labels to match those of our model column labels
new_obs.plot(cmap='Set2')
new_obs = new_obs.rename(columns={"Alpha":"alpha", "Delta":"delta", "Omicron":"omicron", "BA.2":"ba2",
                                  "BA.2.12.1":"ba2121","Other":"wildtype", "Omicron BA.4/5":"ba45", "Scrabble": "scrabble"})
# Similarly, make the column order alphabetical like in our modeled dataframe
new_obs = new_obs.reindex(columns=["alpha","ba2","ba2121","ba45","delta","omicron","scrabble", "wildtype"])
# Create data label column that signifies each entry is observed data - this is useful for when we concat
# this dataset with the modeled dataset
new_obs['data_label'] = ['obs']*len(new_obs['alpha'])
# Make the date into a column instead of the index. This just makes plotting easier later on
new_obs['date'] = new_obs.index

In [ ]:
new_obs.plot(cmap='Set2')

In [ ]:
new_df.plot(cmap='Set2')

In [ ]:
# Now, we make our new dataframe for modeled variant prevalence (new_df)

# Drop the 'none' column since we have nothing to compare this to in our observed data
#new_df = new_df.drop(columns=["none"])
new_df['data_label'] = ['modeled']*len(new_df['alpha'])
new_df['date'] = new_df.index
new_df.info()

In [ ]:
# Concatenate our dataframes to make plotting with the 'hue' seaborn feature simple
all_data = pd.concat([new_df, new_obs], ignore_index=True)

In [ ]:
# Set seaborn plotting template
sns.set_style('darkgrid')
sns.set_context("paper")

In [ ]:
# Set other specifics about plotting like color palette and legend texts
colors = ['peru', 'cadetblue']
sns.set_palette(sns.color_palette(colors))
legend_texts = ['Modeled', 'Observed']

In [ ]:
# Plot and save figure for each variant comparison
# ***Note: This will not show the plots in the jupyter notebook, but rather output them to you *main* directory
for col in all_data:
    plot = sns.lineplot(data=all_data, x='date', y=col, hue="data_label", palette=colors)
    plot.set(xlabel='Date', ylabel='Variant Prevalence (Normalized)')
    plot.set_title(f'{col} variant share')
    h,l = plot.get_legend_handles_labels()
    l = legend_texts
    plot.legend(h,l, loc=0, borderaxespad=0.,title='Data Source' ,fontsize=10)
    plot.get_figure().savefig(f"{col}_variant_share.png", dpi=300, bbox_inches = "tight")
    plt.clf()

In [ ]:
obs.plot(cmap='Set2')
new_df.plot(cmap='Set2')